# Functions

In [ ]:
import numpy as np
from mpmath import mp
from tqdm import tqdm
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
from utils import save_exponents_csv
#from renormalization_cell123_mp import *

mp.dps = 100  # Set desired precision

plus_configs = [
    ([1,1,1], lambda J1, J2: 2*J1 + J2, mp.mpf('3')),
    ([1,1,-1], lambda J1, J2: -J2, mp.mpf('1')),
    ([1,-1,1], lambda J1, J2: -2*J1 + J2, mp.mpf('1')),
    ([-1,1,1], lambda J1, J2: -J2, mp.mpf('1'))
]

minus_configs = [
    ([-1,-1,-1], lambda J1, J2: 2*J1 + J2, mp.mpf('-3')),
    ([-1,-1,1], lambda J1, J2: -J2, mp.mpf('-1')),
    ([-1,1,-1], lambda J1, J2: -2*J1 + J2, mp.mpf('-1')),
    ([1,-1,-1], lambda J1, J2: -J2, mp.mpf('-1'))
]

def get_J(d, J0, n):
    if d <= 0:
        return mp.mpf(0)
    return mp.mpf(J0) / mp.power(mp.mpf(d), mp.mpf(n))

def mp_logsumexp(lst):
    if not lst:
        return mp.ninf
    a_max = max(lst)
    if mp.isinf(a_max):
        return a_max
    tmp = sum(mp.exp(a - a_max) for a in lst)
    return a_max + mp.log(tmp)

def compute_R_logs(start, J_func, h):
    J1 = J_func(1)
    J2 = J_func(2)
    distances = [(iL, iR, (start + iR) - (1 + iL)) for iL in range(3) for iR in range(3)]

    def get_totals(left_confs, right_confs):
        totals = []
        for spinsL, El_func, sumL in left_confs:
            El = El_func(J1, J2)
            for spinsR, Er_func, sumR in right_confs:
                Er = Er_func(J1, J2)
                Eint = mp.mpf('0')
                for iL, iR, d in distances:
                    Eint += mp.mpf(spinsL[iL] * spinsR[iR]) * J_func(d)
                total = El + Er + Eint + h * (sumL + sumR)
                totals.append(total)
        return totals

    totals_pp = get_totals(plus_configs, plus_configs)
    totals_pm = get_totals(plus_configs, minus_configs)
    totals_mp = get_totals(minus_configs, plus_configs)
    totals_mm = get_totals(minus_configs, minus_configs)

    log_R_pp = mp_logsumexp(totals_pp)
    log_R_pm = mp_logsumexp(totals_pm)
    log_R_mp = mp_logsumexp(totals_mp)
    log_R_mm = mp_logsumexp(totals_mm)

    return log_R_pp, log_R_pm, log_R_mp, log_R_mm

def compute_J_prime_func(start, J_func, h):
    log_pp, log_pm, log_mp, log_mm = compute_R_logs(start, J_func, h)
    if mp.isinf(log_pm) or mp.isinf(log_mp):
        return mp.inf
    ln_arg = log_pp + log_mm - log_pm - log_mp
    return mp.mpf('0.25') * ln_arg

def compute_H_prime_func(start, J_func, h): # start = 4 for r'=1
    log_pp, _, _, log_mm = compute_R_logs(start, J_func, h)
    ln_arg = log_pp - log_mm
    return mp.mpf('0.25') * ln_arg

def generate_rg_flow(J0, n, max_k, num_steps):
    J_func = lambda d: get_J(d, J0, n) if d > 0 else mp.mpf(0)
    rs = list(range(1, max_k + 1))
    initial_Js = [J_func(r) for r in rs]
    all_Js = [initial_Js]
    all_J_funcs = [J_func]
    h = mp.mpf(0)
    for step in range(1, num_steps + 1):
        Jps = []
        for r in rs:
            start = 3 * r + 1
            Jp = compute_J_prime_func(start, J_func, h)
            Jps.append(Jp)
        all_Js.append(Jps)
        J_dict = {r: Jps[r-1] for r in rs}
        J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
        all_J_funcs.append(J_func)
    return all_Js, all_J_funcs

def construct_recursion_matrix(J0, n, max_k, num_steps, matrix_size=5, epsilon=1e-6):
    all_Js, _ = generate_rg_flow(J0, n, max_k, num_steps)
    if len(all_Js) < 2:
        raise ValueError("Need at least 2 steps for derivatives")

    # Extract first matrix_size from last two steps
    J_k_minus_1 = all_Js[-2][:matrix_size]  # list of mp.mpf, truncated
    J_k = all_Js[-1][:matrix_size]  # list of mp.mpf, truncated

    M = matrix_size  # Size of matrix
    jac = np.zeros((M, M))

    # Define compute_T truncated to M
    def compute_T(J_vec_mp):
        J_dict = {r+1: J_vec_mp[r] for r in range(M)}
        J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
        J_prime = []
        h = mp.mpf(0)  # h=0
        for r in range(1, M+1):
            start = 3 * r + 1
            Jp = compute_J_prime_func(start, J_func, h)
            J_prime.append(Jp)
        return J_prime

    epsilon_mp = mp.mpf(epsilon)

    J_k_minus_1_float = np.array([float(j) for j in J_k_minus_1])
    J_k_float = np.array([float(j) for j in J_k])

    for s in range(M):
        J_pert_mp = [j for j in J_k_minus_1]  # copy list of mp
        J_pert_mp[s] += epsilon_mp
        T_pert_mp = compute_T(J_pert_mp)
        T_pert_float = np.array([float(j) for j in T_pert_mp])
        jac[:, s] = (T_pert_float - J_k_float) / float(epsilon)

    return jac

def find_J_c(n, max_k=1000, tol=1e-6, J_low=1e-10, J_high=3.0):
    if n <= 0 or n >= 2:
        raise ValueError("n must be between 0 and 2, excluding the edges.")

    start_track = 3
    max_steps = 5
    tol = mp.mpf(tol)
    J_low = mp.mpf(J_low)
    J_high = mp.mpf(J_high)
    rs = list(range(1, max_k + 1))

    def compute_flow(J0):
        J_func = lambda d: get_J(d, J0, n) if d > 0 else mp.mpf(0)
        all_Js = [[J_func(r) for r in rs]]
        h = mp.mpf(0)
        for step in range(1, max_steps + 1):
            Jps = [compute_J_prime_func(3 * r + 1, J_func, h) for r in rs]
            all_Js.append(Jps)
            if step >= start_track:
                J_r2_current = Jps[1]
                J_r2_previous = all_Js[-2][1]
                if J_r2_current > J_r2_previous:
                    return all_Js, True  # Growing (ferromagnetic)
                if J_r2_current < J_r2_previous:
                    return all_Js, False  # Decaying (paramagnetic)
            J_dict = {r: Jps[r-1] for r in rs}
            J_func = lambda d: J_dict.get(d, mp.mpf(0)) if d > 0 else mp.mpf(0)
        # Fallback
        J_r2_initial = all_Js[0][1]
        J_r2_final = all_Js[-1][1]
        return all_Js, J_r2_final > J_r2_initial

    iter_count = 0
    while J_high - J_low > tol and iter_count < 100:
        iter_count += 1
        J_mid = (J_low + J_high) / 2
        _, is_growing = compute_flow(J_mid)
        if is_growing:
            J_high = J_mid  # Growing: search lower half
        else:
            J_low = J_mid  # Decaying: search upper half
    return (J_low + J_high) / 2

def compute_dH_prime_dH(J0, n, max_k, num_steps, h_small='1e-30'):
    """
    Compute the derivative ∂H'/∂H at low temperature using finite differences.
    
    Parameters:
    J0 (float): Initial coupling strength (high for low temperature)
    n (float): Power-law exponent
    max_k (int): Maximum distance for couplings
    num_steps (int): Number of RG steps to reach fixed point
    h_small (str or float): Small magnetic field perturbation
    
    Returns:
    float: Approximate ∂H'/∂H
    """
    # Generate RG flow to get fixed-point couplings
    all_Js, _ = generate_rg_flow(J0, n, max_k, num_steps)
    J_fixed = all_Js[-1]  # Take the last step as fixed point
    J_func_fixed = lambda d: J_fixed[d-1] if 1 <= d <= max_k else mp.mpf(0)
    
    # Compute H' with small perturbation h_small
    h_small = mp.mpf(h_small)
    H_prime = compute_H_prime_func(4, J_func_fixed, h_small)  # r'=1, start=4
    dH_prime_dH = H_prime / h_small
    
    return float(dH_prime_dH)

def compute_magnetization(J0, n, max_k, num_steps, h_small=mp.mpf('1e-30'), b=3, d=1, M_k=1.0):
    all_Js, all_J_funcs = generate_rg_flow(J0, n, max_k, num_steps)
    product = mp.mpf('1')
    for i in range(num_steps):
        J_func_i = all_J_funcs[i]
        H_prime = compute_H_prime_func(4, J_func_i, h_small)
        dH_dH_i = H_prime / h_small
        product *= dH_dH_i
    M_0 = float(product / mp.power(mp.mpf(b), mp.mpf(num_steps * d)) * M_k)
    return M_0

# Critical Exponents

## Single calculation

In [ ]:
n = 1  # Exponent n
max_k = 1000  # Max distance for flow
num_steps = 5  # Number of RG steps
matrix_size = 5  # Number of first J values for matrix
b = 3.0  # Scale factor

# Find critical Jc
Jc = find_J_c(n, max_k=max_k, tol=1e-6, J_low=1e-10, J_high=3.0)
print(f"Critical Jc for n={n}: {float(Jc)}")

# Compute recursion matrix at Jc
M_matrix = construct_recursion_matrix(Jc, n, max_k, num_steps, matrix_size=matrix_size)
print("Recursion Matrix M:")
print(M_matrix)

# Compute eigenvalues
eigenvalues = np.linalg.eigvals(M_matrix)
eigenvalues = sorted(eigenvalues, key=abs, reverse=True)
lambda_t = eigenvalues[0]
yT = np.log(abs(lambda_t)) / np.log(b) if abs(lambda_t) > 0 else 0
nu = 1 / yT if yT != 0 else np.inf
alpha = 2 - 1 / yT if yT != 0 else -np.inf
print(f"Largest eigenvalue λ_t = {lambda_t:.6f}")
print(f"ν = {nu:.6f}, α = {alpha:.6f}")

# Compute yH, eta, delta
all_Js, _ = generate_rg_flow_no_viz(Jc, n, max_k, num_steps)
J_fixed = all_Js[-1]
J_func_fixed = lambda d: J_fixed[d-1] if 1 <= d <= max_k else mp.mpf(0)

h_small = mp.mpf('1e-30')
H_prime = compute_H_prime_func(4, J_func_fixed, h_small)
dyh = H_prime / h_small
yH = float(mp.log(dyh) / mp.log(mp.mpf('3')))

eta = 1.0 - yH
delta = yT / (1.0 - yH)

print(f"yH = {yH:.6f}")
print(f"η = {eta:.6f}")
print(f"δ = {delta:.6f}")

## Multiple calculations

In [ ]:
import numpy as np
from mpmath import mp
import matplotlib.pyplot as plt

# Load the saved data
critical_temp_array = np.load('../data/critical_interaction_data.npy')
n_values = critical_temp_array[:,0]
Jcs = critical_temp_array[:,1]
max_k = 2000
num_steps = 2
matrix_size = 5
b = 3.0
h_small = mp.mpf('1e-30')

# Lists to store exponents
nus = []
alphas = []
etas = []
deltas = []
betas = []
gammas = []

for n, Jc in zip(n_values, Jcs):
    print(f"Processing n = {n:.2f}")
   
    # Compute recursion matrix at Jc
    M_matrix = construct_recursion_matrix(Jc, n, max_k, num_steps, matrix_size=matrix_size)
   
    # Compute eigenvalues for thermal exponents
    eigenvalues = np.linalg.eigvals(M_matrix)
    eigenvalues = sorted(eigenvalues, key=abs, reverse=True)
    lambda_t = eigenvalues[0]
    yT = np.log(abs(lambda_t)) / np.log(b) if abs(lambda_t) > 0 else 0
    nu = 1 / yT if yT != 0 else np.inf
    alpha = 2 - 1 / yT if yT != 0 else -np.inf
    nus.append(nu)
    alphas.append(alpha)
    print(f" ν = {nu:.6f}, α = {alpha:.6f}")
   
    # Compute magnetic exponents
    all_Js, _ = generate_rg_flow(Jc, n, max_k, num_steps)
    J_fixed = all_Js[-1]
    J_func_fixed = lambda d: J_fixed[d-1] if 1 <= d <= max_k else mp.mpf(0)
   
    H_prime = compute_H_prime_func(4, J_func_fixed, h_small)
    dyh = H_prime / h_small
    yH = float(mp.log(dyh) / mp.log(mp.mpf('3')))
   
    eta = 2 + 1 - 2 * yH  # Standard formula for d=1: η = 2 + d - 2 yH = 3 - 2 yH (corrected to match known cases)
    delta = yH / (1.0 - yH)  # Corrected to standard δ = yH / (d - yH) for d=1
    beta = (1.0 - yH) / yT  # Added β = (d - yH) / yT for d=1
    gamma = (2 - eta) * nu if np.isfinite(nu) else np.inf  # Added γ = (2 - η) ν
    etas.append(eta)
    deltas.append(delta)
    betas.append(beta)
    gammas.append(gamma)
    print(f" η = {eta:.6f}, δ = {delta:.6f}, β = {beta:.6f}, γ = {gamma:.6f}")

# Plot exponents vs n
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
axs[0,0].plot(n_values, nus, marker='o')
axs[0,0].set_xlabel('n')
axs[0,0].set_ylabel('ν')
axs[0,1].plot(n_values, alphas, marker='o')
axs[0,1].set_xlabel('n')
axs[0,1].set_ylabel('α')
axs[1,0].plot(n_values, etas, marker='o')
axs[1,0].set_xlabel('n')
axs[1,0].set_ylabel('η')
axs[1,1].plot(n_values, deltas, marker='o')
axs[1,1].set_xlabel('n')
axs[1,1].set_ylabel('δ')
axs[2,0].plot(n_values, betas, marker='o')
axs[2,0].set_xlabel('n')
axs[2,0].set_ylabel('β')
axs[2,1].plot(n_values, gammas, marker='o')
axs[2,1].set_xlabel('n')
axs[2,1].set_ylabel('γ')
plt.tight_layout()
plt.show()

## Plot and save data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
from utils import save_exponents_csv

# Plot the results
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
axs[0, 0].plot(n_values, nus, marker='o', label='ν')
axs[0, 0].set_xlabel('a')
axs[0, 0].set_ylabel('ν')
axs[0, 0].grid(True)
axs[0, 0].legend()

axs[0, 1].plot(n_values, alphas, marker='o', label='α')
axs[0, 1].set_xlabel('a')
axs[0, 1].set_ylabel('α')
axs[0, 1].grid(True)
axs[0, 1].legend()

axs[1, 0].plot(n_values, etas, marker='o', label='η')
axs[1, 0].set_xlabel('a')
axs[1, 0].set_ylabel('η')
axs[1, 0].grid(True)
axs[1, 0].legend()

axs[1, 1].plot(n_values, deltas, marker='o', label='δ')
axs[1, 1].set_xlabel('a')
axs[1, 1].set_ylabel('δ')
axs[1, 1].grid(True)
axs[1, 1].legend()

axs[2, 0].plot(n_values, betas, marker='o', label='β')
axs[2, 0].set_xlabel('a')
axs[2, 0].set_ylabel('β')
axs[2, 0].grid(True)
axs[2, 0].legend()

axs[2, 1].plot(n_values, gammas, marker='o', label='γ')
axs[2, 1].set_xlabel('a')
axs[2, 1].set_ylabel('γ')
axs[2, 1].grid(True)
axs[2, 1].legend()

plt.suptitle('Critical Exponents vs. Interaction Exponent a')
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust for suptitle

# Save the plot
os.makedirs("../figures", exist_ok=True)
#plt.savefig("../figures/all_exponents.png", dpi=300, bbox_inches='tight')
print("Plot saved to ../figures/all_exponents.png")
plt.show()

In [ ]:
# Save the exponents data
save_exponents_csv(n_values, Jcs, nus, alphas, etas, deltas, betas, gammas, filename="../data/exponents_all.csv")

# Derivative of H

In [ ]:
n = 1  # Power-law exponent
max_k = 3**7  # Max distance for flow
num_steps = 7  # Number of RG steps
h_small = '1e-30'  # Small perturbation for numerical derivative

J = find_J_c(n, max_k=max_k, tol=1e-4, J_low=1e-4, J_high=3.0)
dH_prime_dH = compute_dH_prime_dH(J, n, max_k, num_steps, h_small)
print(f"∂H'/∂H at J0={float(J):.4f} for n={n}: {dH_prime_dH:.6f}")

In [ ]:
max_k = 3**7  # Max distance for flow
num_steps = 7  # Number of RG steps
h_small = '1e-30'  # Small perturbation for numerical derivative

a_values = np.linspace(1, 2, 20)
dH_values = []

for i, n in enumerate(a_values):
    J = find_J_c(n, max_k=max_k, tol=1e-4, J_low=1e-4, J_high=3.0)
    dH_prime_dH = compute_dH_prime_dH(J, n, max_k, num_steps, h_small)
    dH_values.append(dH_prime_dH)
    print(f"∂H'/∂H at J0={float(J):.4f} for n={n}: {dH_prime_dH:.6f}")

plt.plot(a_values, dH_values)

In [ ]:
res = np.array([[1.0             , 1.823854],
[1.0526315789473684, 1.904135],
[1.1052631578947367, 1.990792],
[1.1578947368421053, 2.085241],
[1.2105263157894737, 2.184977],
[1.263157894736842, 2.288781],
[1.3157894736842106, 2.399111],
[1.368421052631579, 2.501217],
[1.4210526315789473, 2.602921],
[1.4736842105263157, 2.690388],
[1.526315789473684, 2.766802],
[1.5789473684210527, 2.830759],
[1.631578947368421, 2.879554],
[1.6842105263157894, 2.918329],
[1.736842105263158, 2.946860],
[1.7894736842105263, 2.967462],
[1.8421052631578947, 2.982446],
[1.894736842105263, 2.992495],
[1.9473684210526314, 2.998302]])

plt.figure(figsize=(8, 6))
plt.plot(res[:,0], res[:,1])
plt.xlabel('a')
plt.ylabel('∂H\'/∂H at Tc')
#plt.xlim(0, max(T_values))
#plt.set_ylim(0, 1.05)
plt.grid(True)
#plt.savefig(f'../figures/derivativeH_n{n}.png')
plt.show()

In [ ]:
Jc = find_J_c(n, max_k=max_k, tol=1e-6, J_low=1e-10, J_high=3.0)
Tc = 1 / float(Jc)

In [ ]:
# Parameters
n = 1.5
max_k = 100000
num_steps = 8
h_small = mp.mpf('1e-30')

# Compute dH'_i/dH_i for each T
T_values = np.linspace(5, 8.0, 5)
dH_dH_values = []
for T in T_values:
    J0 = 1.0 / T
    dH_prime_dH = compute_dH_prime_dH(J0, n, max_k, num_steps, h_small)
    dH_dH_values.append(float(dH_prime_dH))

# Plot
plt.figure(figsize=(8, 6))
plt.plot(T_values, dH_dH_values, label=f'n={n}', marker='o')
plt.xlabel('Temperature T')
plt.ylabel('∂H\'/∂H (5th RG step)')
#plt.title('∂H\'/∂H vs T for n=1')
plt.axvline(x=Tc, color='r', linestyle='--', label=f'$T_c$ ≈ {Tc:.3f}')
plt.xlim(0, max(T_values))
#plt.set_ylim(0, 1.05)
plt.legend()
plt.grid(False)
#plt.savefig(f'../figures/derivativeH_n{n}.png')
plt.show()

In [ ]:
# Plot
plt.figure(figsize=(8, 6))
plt.plot(T_values, dH_dH_values, label=f'n={n}', marker='o')
plt.xlabel('Temperature T')
plt.ylabel('∂H\'/∂H (5th RG step)')
#plt.title('∂H\'/∂H vs T for n=1')
plt.axvline(x=Tc, color='r', linestyle='--', label=f'$T_c$ ≈ {Tc:.3f}')
plt.xlim(0, max(T_values))
#plt.set_ylim(0, 1.05)
plt.legend()
plt.grid(False)
#plt.savefig(f'../figures/derivativeH_n{n}.png')
plt.show()